## Build Speech data files

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display

%matplotlib inline

kode ini merubah dari 7 kelas emosi menjadi 5 kelas emosi

In [33]:
df = pd.read_csv('data/pre-processed/audio_features.csv')
df = df[df['label'].isin([0, 1, 2, 3, 4, 5, 6, 7])]
print(df.shape)
display(df.head())

# change 7 to 2
df['label'] = df['label'].map({0: 0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 4, 7: 5})
df.head()

(1402, 10)


,wav_file,label,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std
0,Ses01F_impro01_F000,7,0.004029,0.007451,0.005045,0.005466,0.125000,-0.007787,0.043667,0.165918
1,Ses01F_impro01_F001,7,0.004209,0.009823,0.005309,0.008218,0.491667,-0.006072,0.083406,0.160758
2,Ses01F_impro01_F002,7,0.003928,0.007829,0.004941,0.006064,0.385185,-0.010218,0.049978,0.112936
5,Ses01F_impro01_F005,7,0.030085,0.054550,0.037255,0.039751,0.428161,-0.005372,2.280359,7.328892
6,Ses01F_impro01_F006,4,0.046330,0.074601,0.056799,0.048212,0.255639,-0.009276,3.977544,21.287299


,wav_file,label,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std
0,Ses01F_impro01_F000,5,0.004029,0.007451,0.005045,0.005466,0.125000,-0.007787,0.043667,0.165918
1,Ses01F_impro01_F001,5,0.004209,0.009823,0.005309,0.008218,0.491667,-0.006072,0.083406,0.160758
2,Ses01F_impro01_F002,5,0.003928,0.007829,0.004941,0.006064,0.385185,-0.010218,0.049978,0.112936
5,Ses01F_impro01_F005,5,0.030085,0.054550,0.037255,0.039751,0.428161,-0.005372,2.280359,7.328892
6,Ses01F_impro01_F006,2,0.046330,0.074601,0.056799,0.048212,0.255639,-0.009276,3.977544,21.287299


In [34]:
print(dict(pd.Series(df['label']).value_counts()))

{2: 474, 5: 384, 1: 278, 0: 229, 4: 25, 3: 12}


In [35]:
df.to_csv('data/no_sample_df.csv')

# oversample fear
df = pd.concat([df] + [fear_df]*30 + [sur_df]*10, ignore_index=True)




df.to_csv('data/modified_df.csv')
df.head()
df.tail()

,wav_file,label,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std
2007,Ses01M_script01_1_F034,4,0.014131,0.031626,0.017998,0.025950,0.630631,-0.012647,0.847325,1.681217
2008,Ses01M_script02_1_F002,4,0.007660,0.016159,0.009945,0.012715,0.600000,-0.012433,0.217545,0.296042
2009,Ses01M_script02_2_F038,4,0.017675,0.034532,0.022097,0.026530,0.524964,-0.007070,0.953973,3.062275
2010,Ses01M_script02_2_F039,4,0.017598,0.035494,0.021909,0.027922,0.555825,-0.012053,1.021603,3.191883
2011,Ses01M_script03_2_F003,4,0.052026,0.095277,0.065705,0.068909,0.440506,-0.014940,7.032692,15.334435


In [36]:
print(dict(pd.Series(df['label']).value_counts()))

{2: 474, 5: 384, 3: 372, 1: 278, 4: 275, 0: 229}


In [37]:
df.tail()

,wav_file,label,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std
2007,Ses01M_script01_1_F034,4,0.014131,0.031626,0.017998,0.025950,0.630631,-0.012647,0.847325,1.681217
2008,Ses01M_script02_1_F002,4,0.007660,0.016159,0.009945,0.012715,0.600000,-0.012433,0.217545,0.296042
2009,Ses01M_script02_2_F038,4,0.017675,0.034532,0.022097,0.026530,0.524964,-0.007070,0.953973,3.062275
2010,Ses01M_script02_2_F039,4,0.017598,0.035494,0.021909,0.027922,0.555825,-0.012053,1.021603,3.191883
2011,Ses01M_script03_2_F003,4,0.052026,0.095277,0.065705,0.068909,0.440506,-0.014940,7.032692,15.334435


In [38]:

unique_values = df['label'].unique()
print(unique_values)


[5 2 0 1 4 3]


In [39]:
import numpy as np

# Ambil 5 data pertama
first_five =  df.head(5)

# Nilai minimal dan maksimal per kolom
min_vals = np.min(first_five, axis=0)
max_vals = np.max(first_five, axis=0)

print("Nilai minimal per kolom pada 5 data pertama:", min_vals)
print("Nilai maksimal per kolom pada 5 data pertama:", max_vals)


Nilai minimal per kolom pada 5 data pertama: wav_file         Ses01F_impro01_F000
label                              2
sig_mean                    0.003928
sig_std                     0.007451
rmse_mean                   0.004941
rmse_std                    0.005466
silence                        0.125
harmonic                   -0.010218
auto_corr_max               0.043667
auto_corr_std               0.112936
dtype: object
Nilai maksimal per kolom pada 5 data pertama: wav_file         Ses01F_impro01_F006
label                              5
sig_mean                     0.04633
sig_std                     0.074601
rmse_mean                   0.056799
rmse_std                    0.048212
silence                     0.491667
harmonic                   -0.005372
auto_corr_max               3.977544
auto_corr_std              21.287299
dtype: object


In [40]:

unique_values = df['label'].unique()
print(unique_values)


[5 2 0 1 4 3]


In [41]:
import numpy as np
df = pd.read_csv('data/modified_df.csv')

emotion_dict = {'ang': 0,
                'hap': 1,
                'sad': 2,
                'neu': 3,}

# emotion_dict = {'ang': 0,
#                 'hap': 1,
#                 'exc': 2,
#                 'sad': 3,
#                 'fru': 4,
#                 'fea': 5,
#                 'sur': 6,
#                 'neu': 7,
#                 'xxx': 8,
#                 'oth': 8}


scalar = MinMaxScaler()
df[df.columns[3:]] = scalar.fit_transform(df[df.columns[3:]])
df.head()

,Unnamed: 0,wav_file,label,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std
0,0,Ses01F_impro01_F000,5,0.008218,0.014212,0.008571,0.028135,0.166244,0.171332,0.000498,0.000563
1,1,Ses01F_impro01_F001,5,0.008853,0.020548,0.009339,0.043256,0.653892,0.171667,0.000979,0.000545
2,2,Ses01F_impro01_F002,5,0.007861,0.015221,0.008268,0.031424,0.512277,0.170858,0.000575,0.000376
3,3,Ses01F_impro01_F005,5,0.100066,0.140018,0.102109,0.216499,0.569432,0.171803,0.027564,0.025883
4,4,Ses01F_impro01_F006,2,0.157329,0.193577,0.158861,0.262985,0.339987,0.171042,0.048101,0.075224


In [42]:

unique_values = df['label'].unique()
print(unique_values)

unique_count = df['label'].nunique()
print(f"Jumlah nilai unik di kolom 'nama_kolom': {unique_count}")



[5 2 0 1 4 3]
Jumlah nilai unik di kolom 'nama_kolom': 6


In [43]:
x_train, x_test = train_test_split(df, test_size=0.20)

x_train.to_csv('data/s2e/audio_train.csv', index=False)
x_test.to_csv('data/s2e/audio_test.csv', index=False)

print(x_train.shape, x_test.shape)

(1609, 11) (403, 11)


In [44]:
print(dict(pd.Series(x_train['label']).value_counts()))

{2: 369, 5: 318, 3: 293, 1: 227, 4: 218, 0: 184}


## Define preprocessing functions for text

In [45]:
import unicodedata

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

## Build Text data files

In [46]:
import re
import os
import pickle

useful_regex = re.compile(r'^(\w+)', re.IGNORECASE)

file2transcriptions = {}

for sess in [1]:
    transcripts_path = 'data/IEMOCAP_full_release/Session{}/dialog/transcriptions/'.format(sess)
    transcript_files = os.listdir(transcripts_path)
    for f in transcript_files:
        with open('{}{}'.format(transcripts_path, f), 'r') as f:
            all_lines = f.readlines()

        for l in all_lines:
            audio_code = useful_regex.match(l).group()
            transcription = l.split(':')[-1].strip()
            # assuming that all the keys would be unique and hence no `try`
            file2transcriptions[audio_code] = transcription
# save dict
with open('data/t2e/audiocode2text.pkl', 'wb') as file:
    pickle.dump(file2transcriptions, file)
len(file2transcriptions)

1821

In [47]:
audiocode2text = pickle.load(open('data/t2e/audiocode2text.pkl', 'rb'))

In [48]:
# Prepare text data
text_train = pd.DataFrame()
text_train['wav_file'] = x_train['wav_file']
text_train['label'] = x_train['label']
text_train['transcription'] = [normalizeString(audiocode2text[code]) for code in x_train['wav_file']]

text_test = pd.DataFrame()
text_test['wav_file'] = x_test['wav_file']
text_test['label'] = x_test['label']
text_test['transcription'] = [normalizeString(audiocode2text[code]) for code in x_test['wav_file']]

text_train.to_csv('data/t2e/text_train.csv', index=False)
text_test.to_csv('data/t2e/text_test.csv', index=False)

print(text_train.shape, text_test.shape)

(1609, 3) (403, 3)
